## Decorators
The decorator replaces the function

In [ ]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()
print(target)


### When are decorators executed by python
Decorators are executed immediately when a module is loaded, they run right after the decorated function is defined. <br>
Decorators are imported at import time, the decorated functions are running at runtime.

In [ ]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1')

@register
def f2():
    print('running f2')

def f3():
    print('running f3')

def main():
    print('running main')
    print('registry ->', registry)

    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()
    

### Decorator enhanced strategy pattern

In [ ]:
# My noob implementation

from typing import Callable

promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

class Order:
    def __init__(self, price: int, strategies: list[Callable]):
        self.price = price
        self.strategies = strategies

    def pay(self):
        return min(strategy(self.price) for strategy in self.strategies)
    
@promotion
def happy_deal(price):
    return price*0.75

@promotion
def winter_sale(price):
    return price*0.50


order = Order(34, promos)
payment = order.pay()
print(payment)


In [ ]:
# Fluent python pro solution

promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total()*.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total()*.1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total()*.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

### Variable scope rules

In [ ]:
# function reading a local and a global variable
def f1(a):
    print(a)
    print(b)

f1(3)

In [ ]:
b=6
f1(3)

Inside of a function Python decides if a variable is local or global based on the fact if the variable is assigned inside of the function, regardless of the fact that a global variable with the same name exists or not.

In [ ]:
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

If you want to use b as a global variable use global

In [ ]:
b=6

def f3(a):
    global b 
    print(a)
    print(b)
    b=9

f3(2)
print(b)
b=7
print(b)

## Closures
The idea is to have a function/method wich makes use of nonglobal variables which are not defined in the scope of the function/method. <br>
This can be achieved bu using a callable class or a higher order function.

In [7]:
# Example with classes

class Averager:
    def __init__(self):
        self.historical_data = []
    
    def __call__(self, new_data):
        self.historical_data.append(new_data)
        return sum(self.historical_data)/len(self.historical_data)

avg = Averager()
print(avg(3))
print(avg(7))

3.0
5.0


In the implementation with the higher order function (the closure), the variable *historical_data* is a **free variable**, this means that is a variable which is not bound in the local scope. <br>

In [ ]:
# Example with higher order function

def make_averager():
    historical_data = []
    def averager(new_data):
        historical_data.append(new_data)
        return sum(historical_data)/len(historical_data)
    return averager

avg = make_averager()
print(avg(3))
print(avg(7))

avg2 = make_averager()
print(avg(1))

Names of local and free variables are kept in the `__code__` attribute of the closure object (the "code" attribute represents the compiled body of the function) 

In [ ]:
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

The binding for free variables are kept in the `__closure__` attribute of the closure. Wach item in `__closure__` corresponds to a name in `__code__.co_freevars`

In [ ]:
print(avg.__code__.co_freevars)
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

### Summary closure
A closure is a function that retains the bindings of a free variable which exists when the function was defined. So the free variable can be used later when the function is invoked and the defining scope is no longer available. <br>
Note that the only situation in which a function may need to deal with a variable which is not global, is when the function is nested into another one.

## The `nonlocal` declaration
A more efficient version of the averager to avoid computing the mean every time.

In [ ]:
# Broken implementation --> ints are immutable, if you try to rebind them then the function considers them as local variables

def make_smart_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_smart_averager()
avg(10)

To mark variables as free, you can make use of the `nonlocal` declaration

In [ ]:
# Correct implementation

def make_smart_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_smart_averager()
print(avg(10))
print(avg(20))

avg2 = make_smart_averager()
print(avg2(5))

## Decorators with nested functions

In [ ]:
# Decorator to print running time of the function

import time

def clock(func):

    def clocked(*args):
        start_time = time.perf_counter()
        result = func(*args)
        end_time = time.perf_counter()
        ex_time_ms = round((end_time-start_time)*10**9)
        print(f"Function took {ex_time_ms} [ns] to run")
        return result

    return clocked

@clock
def average(some_list):
    return sum(some_list) / len(some_list)

the_list = range(1, 1000)
print(average(the_list))
print(average.__name__)



## Standard library decorators

### Memoization with `functools.lru_cache`
**Memoization**: an optimization technique thath works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments. <br>
LRU stands for _last recently used_ meaning that elements in the cache wich were not been read for a long time will be removed first.

In [40]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(4)

Function took 400 [ns] to run
Function took 700 [ns] to run
Function took 365300 [ns] to run
Function took 800 [ns] to run
Function took 381900 [ns] to run


3

### Generic functions with single Dispatch
`functools.singledispatch` allows to define generic functions, this is a work around for the fact that there is no overloading of functions and methods in python.

In [ ]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)



### Parameterized Decorators
The idea is to use a decorator factory

In [4]:
registry = set()

def register(active = True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' %(active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

print(registry)
register()(f3)
print(registry)
register(active=False)(f2)
print(registry)

running register(active=False)->decorate(<function f1 at 0x0000023DA4AE14E0>)
running register(active=True)->decorate(<function f2 at 0x0000023DA4AE1440>)
{<function f2 at 0x0000023DA4AE1440>}
running register(active=True)->decorate(<function f3 at 0x0000023DA4A77560>)
{<function f2 at 0x0000023DA4AE1440>, <function f3 at 0x0000023DA4A77560>}
running register(active=False)->decorate(<function f2 at 0x0000023DA4AE1440>)
{<function f3 at 0x0000023DA4A77560>}


### Parametrized nested decorators

In [6]:
# Users may pass a format string to format the output of the clocked function

import time


CUSTOM_TEXT = 'Run time: '

def clock(text=CUSTOM_TEXT):
    format
    def decorator(func):
        def clocked(*args, **kwargs):
            prev_time = time.time()
            result = func(*args, **kwargs)
            run_time = time.time() - prev_time
            print(text, run_time)
            return result
        return clocked
    return decorator

@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(i)

@clock("Custom text: ")
def custom_snooze(seconds):
    time.sleep(seconds)

custom_snooze(1)





Run time:  5.9604644775390625e-06
Run time:  1.0004689693450928
Run time:  2.0001108646392822
Custom text:  1.0005719661712646
